In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
# Session setup
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NYC").getOrCreate()

22/03/25 16:22:51 WARN Utils: Your hostname, futamatatakuminoMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.64 instead (on interface en0)
22/03/25 16:22:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/25 16:22:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# loading csv data
data = spark.read.csv('/Users/futamatatakumi/Documents/Python/RS_project/Parking_Violations_Issued_-_Fiscal_Year_2017.csv', header=True, inferSchema=True)

In [6]:
data.printSchema()

root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Plate Type: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Issuing Agency: string (nullable = true)
 |-- Street Code1: integer (nullable = true)
 |-- Street Code2: integer (nullable = true)
 |-- Street Code3: integer (nullable = true)
 |-- Vehicle Expiration Date: integer (nullable = true)
 |-- Violation Location: integer (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Issuer Code: integer (nullable = true)
 |-- Issuer Command: string (nullable = true)
 |-- Issuer Squad: string (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- Time First Observed: string (nullable = true)
 |-- Violation Coun

In [8]:
# checking shape
shape = (data.count(), len(data.columns))
print('data_shape:', shape)

data_shape: (10803028, 43)


In [15]:
# showing two rows
data.head(2)

[Row(Summons Number=5092469481, Plate ID='GZH7067', Registration State='NY', Plate Type='PAS', Issue Date='07/10/2016', Violation Code=7, Vehicle Body Type='SUBN', Vehicle Make='TOYOT', Issuing Agency='V', Street Code1=0, Street Code2=0, Street Code3=0, Vehicle Expiration Date=0, Violation Location=None, Violation Precinct=0, Issuer Precinct=0, Issuer Code=0, Issuer Command=None, Issuer Squad=None, Violation Time='0143A', Time First Observed=None, Violation County='BX', Violation In Front Of Or Opposite=None, House Number=None, Street Name='ALLERTON AVE (W/B) @', Intersecting Street='BARNES AVE', Date First Observed=0, Law Section=1111, Sub Division='D', Violation Legal Code='T', Days Parking In Effect    =None, From Hours In Effect=None, To Hours In Effect=None, Vehicle Color='GY', Unregistered Vehicle?=None, Vehicle Year=2001, Meter Number=None, Feet From Curb=0, Violation Post Code=None, Violation Description='FAILURE TO STOP AT RED LIGHT', No Standing or Stopping Violation=None, Hy

In [16]:
# general statistics
data.describe().show()

+-------+--------------------+--------+------------------+--------------------+----------+------------------+-----------------+------------------+--------------+-----------------+------------------+------------------+-----------------------+------------------+------------------+-----------------+------------------+--------------------+--------------------+-----------------+-------------------+----------------+---------------------------------+------------+-----------------+-------------------+-------------------+------------------+------------------+--------------------+--------------------------+--------------------+------------------+-------------+---------------------+------------------+------------+-------------------+-------------------+---------------------+---------------------------------+-----------------+------------------------+
|summary|      Summons Number|Plate ID|Registration State|          Plate Type|Issue Date|    Violation Code|Vehicle Body Type|      Vehicle Make|Is

In [17]:
# checking null
data.describe().collect()[0]

Row(summary='count', Summons Number='10803028', Plate ID='10803027', Registration State='10803028', Plate Type='10803028', Issue Date='10803028', Violation Code='10803028', Vehicle Body Type='10760333', Vehicle Make='10729981', Issuing Agency='10803028', Street Code1='10803028', Street Code2='10803028', Street Code3='10803028', Vehicle Expiration Date='10803028', Violation Location='8730628', Violation Precinct='10803028', Issuer Precinct='10803028', Issuer Code='10803028', Issuer Command='8740383', Issuer Squad='8739487', Violation Time='10802965', Time First Observed='840747', Violation County='10763481', Violation In Front Of Or Opposite='8641793', House Number='8514410', Street Name='10799019', Intersecting Street='3367555', Date First Observed='10803028', Law Section='10803028', Sub Division='10802255', Violation Legal Code='2062707', Days Parking In Effect    ='8090612', From Hours In Effect='5352082', To Hours In Effect='5352085', Vehicle Color='10650696', Unregistered Vehicle?=

In [18]:
# checking data types
data.dtypes

[('Summons Number', 'bigint'),
 ('Plate ID', 'string'),
 ('Registration State', 'string'),
 ('Plate Type', 'string'),
 ('Issue Date', 'string'),
 ('Violation Code', 'int'),
 ('Vehicle Body Type', 'string'),
 ('Vehicle Make', 'string'),
 ('Issuing Agency', 'string'),
 ('Street Code1', 'int'),
 ('Street Code2', 'int'),
 ('Street Code3', 'int'),
 ('Vehicle Expiration Date', 'int'),
 ('Violation Location', 'int'),
 ('Violation Precinct', 'int'),
 ('Issuer Precinct', 'int'),
 ('Issuer Code', 'int'),
 ('Issuer Command', 'string'),
 ('Issuer Squad', 'string'),
 ('Violation Time', 'string'),
 ('Time First Observed', 'string'),
 ('Violation County', 'string'),
 ('Violation In Front Of Or Opposite', 'string'),
 ('House Number', 'string'),
 ('Street Name', 'string'),
 ('Intersecting Street', 'string'),
 ('Date First Observed', 'int'),
 ('Law Section', 'int'),
 ('Sub Division', 'string'),
 ('Violation Legal Code', 'string'),
 ('Days Parking In Effect    ', 'string'),
 ('From Hours In Effect', 'str

In [25]:
# removing unnecessary columns
data_select = data[['Registration State', 'Summons Number']]
data_select.head(2)

[Row(Registration State='NY', Summons Number=5092469481),
 Row(Registration State='NY', Summons Number=5092451658)]

In [29]:
#grouping the data by Registration State and counting the number of tickets issued
data_grouped = data_select.groupBy("Registration State").count().sort("count", ascending=False).show(5)

+------------------+-------+
|Registration State|  count|
+------------------+-------+
|                NY|8481061|
|                NJ| 925965|
|                PA| 285419|
|                FL| 144556|
|                CT| 141088|
+------------------+-------+
only showing top 5 rows



Conclusion: From this we can conclude, that majorities tickets were issued to the locals of New York, followed by New Jersey and Pennsylvania